<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/GenAI_Practice/Langwatch/try_dspy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try DsPy for RAG Prompt Optimization

* https://github.com/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/GenAI_Practice/Langwatch/dspy_prompt_optimization_online_dashboard.ipynb
* https://dspy.ai/tutorials/rag/

In [1]:
%%capture --no-stderr
!pip install --upgrade nbformat
%pip install -U --quiet dspy

## Prepare LLM

* `http://20.102.90.50:2017/wiki17_abstracts` provides the sources for retrieval here

In [2]:
import os
import pandas as pd
from getpass import getpass
import dspy
from google.colab import userdata


# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# llm = dspy.LM("openai/gpt-4.1-nano", api_key=OPENAI_API_KEY)

GOOGLE_AI_API_KEY = userdata.get('GOOGLE_AI_API_KEY')
llm = dspy.LM("gemini/gemini-2.0-flash", api_key=GOOGLE_AI_API_KEY)
print("LLM test response:", llm("Where's Silicon Valley?"))

# the retrieval model
colbertv2_wiki17_abstracts = dspy.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)
dspy.settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

LLM test response: ['Silicon Valley is located in the southern part of the San Francisco Bay Area in **Northern California, United States**.\n']


## Preparing Dataset

In [3]:
from dspy.datasets import HotPotQA


dataset = HotPotQA(train_seed=1, train_size=32, eval_seed=2025, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

print()
print(len(trainset), len(devset))
print(trainset[0])
print(devset[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]


32 50
Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})
Example({'question': 'Pehchaan: The Face of Truth stars Vinod Khanna, Rati Agnihotri and which Indian actress, producer, and former model who also produced the film?', 'answer': 'Raveena Tandon', 'gold_titles': {'Pehchaan: The Face of Truth', 'Raveena Tandon'}}) (input_keys={'question'})


## Defining DsPy RAG

In [4]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context,
                               answer=prediction.answer,
                               reasoning=prediction.reasoning)


dev_example = devset[12]
print(f"[Devset] Question: {dev_example.question}")
print(f"[Devset] Answer: {dev_example.answer}")
print(f"[Devset] Relevant Wikipedia Titles: {dev_example.gold_titles}")
print()

generate_answer = RAG()
pred = generate_answer(question=dev_example.question)
print(f"[Prediction] Question: {dev_example.question}")
print(f"[Prediction] Predicted Answer: {pred.answer}")
print(f"[Prediction] Reasoning: {pred.reasoning}")

[Devset] Question: Twelve Inches is a compilation album by which 1980s British band?
[Devset] Answer: Frankie Goes to Hollywood
[Devset] Relevant Wikipedia Titles: {'Frankie Goes to Hollywood', 'Twelve Inches'}

[Prediction] Question: Twelve Inches is a compilation album by which 1980s British band?
[Prediction] Predicted Answer: Soft Cell, Bananarama, Spandau Ballet
[Prediction] Reasoning: The question asks which 1980s British band released a compilation album called "Twelve Inches". I need to find a band that matches both criteria.
The context provides three albums with "Twelve Inches" in the title: "The Twelve Inch Singles" by Soft Cell, "The Twelve Inches of Bananarama" by Bananarama, and "The Twelve Inch Mixes" by Spandau Ballet.
Soft Cell, Bananarama, and Spandau Ballet are all British bands from the 1980s.


## Optimizing Prompts & Logging

In [5]:
from dspy.teleprompt import MIPROv2
import dspy.evaluate


trial_logs = []

# define eval metric
def validate_context_and_answer(example, prediction):
    # basic scoring: exact match (customize as needed)
    gold = example.answer.strip().lower()
    pred = prediction.answer.strip().lower()
    score = int(gold == pred)

    # log this trial
    log_entry = {
        "question": example.question,
        "context": example.context if hasattr(example, "context") else "",
        "ground_truth": gold,
        "prediction": pred,
        "score": score
    }
    trial_logs.append(log_entry)

    # optional: print each trial for real-time tracking
    print(f"[Trial] Q: {example.question} | Pred: {pred} | GT: {gold} | Score: {score}")

    return score


# set up optimizer
optimizer = MIPROv2(metric=validate_context_and_answer, prompt_model=llm,
                    task_model=llm, num_candidates=2, init_temperature=0.7,
                    auto=None)


# compile
compiled_rag = optimizer.compile(
    RAG(),
    trainset=trainset,
    num_trials=10,
    max_bootstrapped_demos=3,
    max_labeled_demos=5,
    minibatch_size=10,
    requires_permission_to_run=False
)

2025/05/25 23:14:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/25 23:14:08 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/25 23:14:08 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=2 sets of demonstrations...
2025/05/25 23:14:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/05/25 23:14:08 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.


Bootstrapping set 1/2
Bootstrapping set 2/2


2025/05/25 23:14:10 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing N=2 instructions...

2025/05/25 23:14:17 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/05/25 23:14:17 INFO dspy.teleprompt.mipro_optimizer_v2: 0: Answer questions with short factoid answers.

2025/05/25 23:14:17 INFO dspy.teleprompt.mipro_optimizer_v2: 1: Answer the question accurately and concisely, using information from the provided context. Provide a brief, step-by-step reasoning to justify your answer. The final answer should be a short factoid.

2025/05/25 23:14:17 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/25 23:14:17 INFO dspy.teleprompt.mipro_optimizer_v2: ==> STEP 3: FINDING OPTIMAL PROMPT PARAMETERS <==
2025/05/25 23:14:17 INFO dspy.teleprompt.mipro_optimizer_v2: We will evaluate the program over a series of trials with different combinations of instructions and few-shot examples to find the optimal combination using Bayesian Optimization.

2025/05/25 23:14

Average Metric: 4.00 / 8 (50.0%):  32%|███▏      | 8/25 [00:02<00:03,  4.97it/s]

2025/05/25 23:14:24 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/25 23:14:24 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/25 23:14:25 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/25 23:14:25 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/25 23:14:25 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/25 23:14:25 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/25 23:14:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/05/25 23:14:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


...

2025/05/25 23:16:35 INFO dspy.evaluate.evaluate: Average Metric: 4 / 10 (40.0%)
2025/05/25 23:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 40.0 on minibatch of size 10 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/05/25 23:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [0.0, 0.0, 0.0, 30.0, 30.0, 0.0, 20.0, 0.0, 40.0]
2025/05/25 23:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [40.0, 36.0]
2025/05/25 23:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 40.0
2025/05/25 23:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/05/25 23:16:35 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 12 / 13 - Minibatch ==



[Trial] Q: On the coast of what ocean is the birthplace of Diogal Sakho? | Pred: atlantic ocean | GT: atlantic | Score: 0
[Trial] Q: Who acted in the shot film The Shore and is also the youngest actress ever to play Ophelia in a Royal Shakespeare Company production of "Hamlet." ? | Pred: kerry condon | GT: kerry condon | Score: 1
[Trial] Q: What person does Wormholes in fiction and Nathan Rosen have in common? | Pred: einstein-rosen bridge | GT: einstein | Score: 0
[Trial] Q: This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what? | Pred: johann strauss ii | GT: the waltz king | Score: 0
[Trial] Q: Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino? | Pred: rosario dawson | GT: rosario dawson | Score: 1
[Trial] Q: Who composed "Sunflower Slow Drag" with the King of Ragtime? | Pred: scott hayden | GT: scott hayden | Score: 1
Average Metric: 6.00 / 10 (60.0%): 

2025/05/25 23:16:36 INFO dspy.evaluate.evaluate: Average Metric: 6 / 10 (60.0%)
2025/05/25 23:16:36 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 60.0 on minibatch of size 10 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/05/25 23:16:36 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [0.0, 0.0, 0.0, 30.0, 30.0, 0.0, 20.0, 0.0, 40.0, 60.0]
2025/05/25 23:16:36 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [40.0, 36.0]
2025/05/25 23:16:36 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 40.0
2025/05/25 23:16:36 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/05/25 23:16:36 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 13 / 13 - Full Evaluation =====
2025/05/25 23:16:36 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 43.333333333333336) from minibatch trials...



[Trial] Q: Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino? | Pred: rosario dawson | GT: rosario dawson | Score: 1
[Trial] Q: What is the code name for the German offensive that started this Second World War engagement on the Eastern Front (a few hundred kilometers from Moscow) between Soviet and German forces, which included 102nd Infantry Division? | Pred: operation citadel | GT: operation citadel | Score: 1
[Trial] Q: Tombstone stared an actor born May 17, 1955 known as who? | Pred: unknown | GT: bill paxton | Score: 0
[Trial] Q: Which company distributed this 1977 American animated film produced by Walt Disney Productions for which Sherman Brothers wrote songs? | Pred: buena vista distribution company | GT: buena vista distribution | Score: 0
[Trial] Q: Samantha Cristoforetti and Mark Shuttleworth are both best known for being first in their field to go where?  | Pred: space | GT: space | Score: 1
[Trial] Q: Who acte

2025/05/25 23:16:38 INFO dspy.evaluate.evaluate: Average Metric: 9 / 25 (36.0%)
2025/05/25 23:16:38 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [40.0, 36.0, 36.0]
2025/05/25 23:16:38 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 40.0
2025/05/25 23:16:38 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/05/25 23:16:38 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/25 23:16:38 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 40.0!


In [10]:
df_logs = pd.DataFrame(trial_logs)
df_logs['trial'] = df_logs.groupby('question').cumcount() + 1
display(df_logs)

,question,context,ground_truth,prediction,score,trial
0,Having the combination of excellent foot speed...,,"""outfield of dreams""",outfield of dreams,0,1
1,Which American actress who made their film deb...,,rosario dawson,rosario dawson,1,1
2,Samantha Cristoforetti and Mark Shuttleworth a...,,space,space,1,1
3,"Tombstone stared an actor born May 17, 1955 kn...",,bill paxton,unknown,0,1
4,What is the code name for the German offensive...,,operation citadel,operation citadel,1,1
...,...,...,...,...,...,...
122,"Who is older, Aleksandr Danilovich Aleksandrov...",,aleksandr danilovich aleksandrov,i do not know.,0,4
123,Who was coach of the No. 9-ranked team that wa...,,fred hoiberg,not mentioned in context,0,4
124,"""Everything Has Changed"" is a song from an alb...",,big machine records,cannot answer.,0,2
125,The Organisation that allows a community to in...,,2010,1992,0,3


In [11]:
df_logs['trial'].value_counts()

,count
trial,
1,25
2,25
3,24
4,23
5,19
6,8
7,1
8,1
9,1
